In [5]:
import os
import json
import numpy as np
import cv2
from pathlib import Path

NB_DIR = Path().resolve()
ROOT = NB_DIR.parent # project root

print("Notebook directory:", NB_DIR)
print("ROOT:", ROOT)

INPUT_DIR = f"{ROOT}/data/lane_data/lane_annotations"
OUTPUT_DIR = f"{ROOT}/data/lane_data/lane_masks"

os.makedirs(OUTPUT_DIR, exist_ok=True)


Notebook directory: /Users/jleto/LocalProjects/rev_vision/notebooks
ROOT: /Users/jleto/LocalProjects/rev_vision


In [6]:
import os
import json
import numpy as np
from PIL import Image
from labelme import utils


for file in os.listdir(INPUT_DIR):
    if not file.endswith(".json"):
        continue

    json_path = os.path.join(INPUT_DIR, file)
    with open(json_path, "r") as f:
        data = json.load(f) # convert json to python dict

    image_data = utils.image.img_b64_to_arr(data["imageData"])
    label_name_to_value = {"_background_": 0} # set unlabeled pixels to 0

    # assign labeled shapes to 1
    for shape in data["shapes"]:
        label_name_to_value[shape["label"]] = 1

    label = utils.shapes_to_label(image_data.shape, data["shapes"], label_name_to_value) # convert polygon vector into 2d label image
    label = (label * 255).astype(np.uint8) # convert to a binary mask

    out_path = os.path.join(OUTPUT_DIR, file.replace(".json", ".png"))
    Image.fromarray(label).save(out_path) # convert numpy array to image
    print(f"Saved: {out_path}")

AttributeError: 'tuple' object has no attribute 'astype'